In [1]:
!pip install pypdf2
!pip install  tabula-py

   ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
   --- ----------------------------------- 20.5/232.6 kB 217.9 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/232.6 kB 393.8 kB/s eta 0:00:01
   -------------------------------------- - 225.3/232.6 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 232.6/232.6 kB 1.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB 653.6 kB/s eta 0:00:19
   - -------------------------------------- 0.3/12.0 MB 3.2 MB/s eta 0:00:04
   --- ------------------------------------ 0.9/12.0 MB 6.4 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/12.0 MB 9.7 MB/s eta 0:00:02
   ---------- ----------------------------- 3.2/12.0 MB 13.7 MB/s eta 0:00:01
   ----------------- ---------------------- 5.2/12.0 MB 18.4 MB/s eta 0:00:01
   ------------------------ --------------- 7.3/12.0 MB 22.1 MB/s eta 0:00:01
   --------------------------------- ------ 10.1/12.0 MB 27.0 MB/s eta 0:00:01
   ---------------------------------------  12.0/12.0 MB 46.7 MB/s eta 0:00:01
   ---------------------------------------  12.0/12.0 MB 46.7 MB/s eta 0:00:01
   ---------------------------------------  12.0/12.0 MB 46.7 MB/s eta 0:00:01



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Vamos aprender a trabalhar com PDF usando o Python

- Regra geral: PDF foi feito justamente para bloquear muita coisa, então não é fácil "brincar" com um pdf
- Mesmo assim, Python tem várias bibliotecas que vão nos ajudar, vamos focar em 2:
    - PyPDF2
    - Tabula
- Ler e extrair informações de um PDF a gente consegue fazer.
- Escrever e Editar, aí já é outra história

### Para os nossos exemplos, vamos avaliar o Release de Resultados do 3º e 4º Trimestre de 2020 da Magazine Luiza

#### 1º Objetivo: Queremos conseguir separar apenas o DRE do Release de Resultados (Página 14) para enviar para a Diretoria, como fazemos?
    - Separar as páginas de um pdf

In [ ]:
import PyPDF2 as pyf

from pathlib import Path

nome = "MGLU_ER_3T20_POR.pdf"
arquivo_pdf = pyf.PdfReader(nome) #vai abrir e ler o arquivo PDF

for i, pagina in enumerate(arquivo_pdf.pages): #para cada indice e pagina no PDF
    num_pagina = i + 1 #pega o nmr da pag/indice do PDF e soma mais um ao percorrer
    novo_pdf = pyf.PdfWriter() #cria um PDF e que pode ser escrito com esse metodo
    novo_pdf.add_page(pagina) #add a pagina do PDF que percorrerá no for

    with Path(f'paginas/Arquivo Pag {num_pagina}.pdf').open(mode='wb') as arquivo: #salva os arquivos na pasta "paginas" e abre o arquivo em modo de escrita
        novo_pdf.write(arquivo) #roda um novo PDF que é o que criamos




#### 2º Objetivo: Com o Release de Resultados já separado página por página, queremos incluir apenas as Páginas de Destaque (Página 1), DRE (Página 14) e Balanço (Página 16).
    - Juntar vários pdfs em 1

In [ ]:
import PyPDF2 as pyf  # Importa a biblioteca PyPDF2 e dá o apelido "pyf" para facilitar o uso

from pathlib import Path  # Importa o Path, que serve para trabalhar com arquivos e caminhos de forma moderna

num_paginas = [1, 14, 16]  # Lista com os números das páginas/arquivos PDF que queremos juntar

novo_arquivo = pyf.PdfWriter()  # Cria um novo PDF vazio, pronto para receber páginas

for num in num_paginas:  # Para cada número na lista de páginas
    pagina_pdf = pyf.PdfReader(f'paginas/Arquivo Pag {num}.pdf')  # Abre e lê o PDF da pasta 'paginas' com o número atual
    novo_arquivo.add_page(pagina_pdf.pages[0])  # Adiciona a primeira (e única) página do PDF lido ao novo PDF

with Path(f'Consolidado.pdf').open(mode='wb') as arquivo:  # Abre (ou cria) o arquivo Consolidado.pdf para escrita binária
    novo_arquivo.write(arquivo)  # Escreve o conteúdo do novo PDF (com as páginas adicionadas) dentro do arquivo


### Extra: Para adicionar todas as páginas de 2 pdfs

In [ ]:
pdf_mesclado = pyf.PdfMerger()

pdf_mesclado.append('MGLU_ER_3T20_POR.pdf')
pdf_mesclado.append('MGLU_ER_4T20_POR.pdf')

with Path(f'Mesclado.pdf').open(mode='wb') as arquivo:
    pdf_mesclado.write(arquivo)

# Funcionalidades que podem ser úteis:

- Inserir arquivo no meio do outro
- Quero colocar dentro do Resultado do 4T20 os destaques do 3T20 para poder comparar os 2 dentro do mesmo relatório

- Rodar Página

# Trabalhando com Textos e Informações Dentro do PDF

#### 1º Objetivo: Quero identificar como foram as Despesas com Vendas da MGLU
    - Pegar texto da página e identificar onde está essa informação

#### 2º Objetivo: Quero analisar o DRE (sem ajuste - Página 5)
    - Para ler tabelas em pdf, use o tabula (é ninja)
    
    - Cuidado 1: Instale o tabula-py (não instale o tabula). Se instalar o tabula errado, desinstale ele, instale o tabula-py, desinstale o tabula-py e instale novamente o tabula-py. Reinicie o kernel do Jupyter após isso
    
    - Cuidado 2: Tem que ter o java instalado no seu computador (depois de instalar, reinicie o computador)

#### 3º Objetivo: Quero analisar o Capital de Giro e os Investimentos (ambas as tabelas na página 12)
    - Páginas com mais de 1 tabela

#### O que fazer quando o tabula não consegue ler alguma linha da tabela? Como o cabeçalho, no nosso caso?

# Outro método que pode ser útil algum dia: Captar Imagem em um pdf
    - biblioteca pikepdf

# Substituir texto no pdf tipo contrato

- Não recomendo fazer diretamente pelo Python. Realmente do que vi a melhor opção me parece o Word fazer isso
- Caso precise automatizar, automatize o processo fazendo ele pelo Word
- Quem quiser MUITO fazer isso pelo Python, tem um link aqui que vai te ajudar de uma solução que achei que funciona. Tem seus bugs/cuidados especiais, mas funciona: https://pdf.co/samples/pdf-co-web-api-replace-text-from-pdf-python-replace-text-from-url